## Sun B1 dynamics: nastier than expected...
When we have 3BP dynamics in E-M rotating frame, we naturally forget about $\omega$ as we normalize t* so that $\omega$ = 1 (i.e., $t_1^*$ = 2$\pi$ / (1 siderial period = 27.5days). 
This means omega should change if we want to discuss the dynamics in Sun-B1 frame ($t_2^*$ = 2$\pi$ / 365 days), while we want to stick with $t_1^*$. Therefore, the angular velocity term in the centrifugal and Coriolis force emerges: $\omega_b = 2\pi$ / (365 days / 27.5 days).

Additionally, according to Boudad's PhD thesis (C.19, P.235), the angular velocity of the E-M line should be $\omega_M = |\omega_S|/(1-|\omega_S|) ~= 1.05$, rather than $\omega_S = - t_{siderial}/t_{synodic} ~= -0.93$, which is the angular velocity of the Sun w.r.t. Earth-Moon rotating frame in BCR4P.




### normalization
$t^* = t_{syderial} = 27.5 * 86400$ [s], 

$l^*$ = Earth-Moon distance,

$m^* = m_E + m_M$.

In [ ]:
using Plots
using LinearAlgebra
using DifferentialEquations
plotly()

In [ ]:
include("../src/SailorMoon.jl")   # relative path to main file of module
include("../../julia-r3bp/R3BP/src/R3BP.jl")

In [ ]:
"""
Right-hand side expression for state-vector in BCR4BP
# Arguments
    - `du`: cache array of duative of state-vector, mutated
    - `u`: state-vector
    - `p`: parameters, where p = [μ2, μS, as, θ0, ωM, τ, γ, β, mdot, tmax]
        m* : mE + mL (6.0455 * 10^22)
        μ2 : mL / m* (0.012150585609624)
        μS : mS / m* (0.00000303951)
        as : (sun-B1 distance) / l* (388.709677419)
            l* : Earth-moon distance (384,400 km)
        ωM : Earth-Moon line's angular velocity around E-M barycenter
        τ  : thrust magnitude (0~1)
        γ  : thrust angle 1
        β  : thrust angle 2
        mdot : mass-flow rate
        tmax : max thrust
    - `t`: time
"""
function rhs_bcr4bp_sb1frame!(du,u,p,t)
    # unpack state
    x, y, z = u[1], u[2], u[3]
    vx, vy, vz = u[4], u[5], u[6]
    μ2, μS, as, θ0, ωM, ωb = p[1], p[2], p[3], p[4], p[5], p[6]
    τ, γ, β, mdot, tmax = p[7], p[8], p[9], p[10], p[11]

    θ = θ0 + ωM * t  # moon angle

    # create Thrust term
    T = dv_inertial_angles([τ,γ,β])
    Tx, Ty, Tz = T * tmax / u[7]  # mdot


    # derivatives of positions
    du[1] = u[4]
    du[2] = u[5]
    du[3] = u[6]

    # earth and moon location
    xe = μS*as/(μS+1) - μ2 *cos(θ)
    ye = μS*as/(μS+1) - μ2 *sin(θ)
    ze = 0

    xm = μS*as/(μS+1) + (1-μ2) *cos(θ)
    ym = μS*as/(μS+1) + (1-μ2) *sin(θ)
    zm = 0

    # compute distances
    r30 = sqrt((-as/(μS+1)- x)^2 + y^2 + z^2)
    r31 = sqrt((xe - x)^2 + (ye - y)^2 + (ze-z)^2)
    r32 = sqrt((xm - x)^2 + (ym - y)^2 + (zm-z)^2)
    println(r30, " ", r31, " ", r32)

    Fx = -(μS)*(x-as/(μS+1))/r30^3 - (1-μ2)*(x-xe)/r31^3 - μ2*(x-xm)/r32^3 + Tx
    Fy = -(μS)*(y-as/(μS+1))/r30^3 - (1-μ2)*(y-ye)/r31^3 - μ2*(y-ym)/r32^3 + Ty
    Fz = -(μS)*(z-as/(μS+1))/r30^3 - (1-μ2)*(z-ze)/r31^3 - μ2*(z-zm)/r32^3 + Tz
    # println(-(μS/(μS+1))*(x-1/(μS+1))/r30^3 , " ", - (1-μ2)/(μS+1)*(x-xe)/r31^3, " ",  - μ2/(μS+1)*(x-xm)/r32^3)


    du[4] =  2*ωb*vy + ωb^2*x + Fx
    du[5] = -2*ωb*vx + ωb^2*y + Fy
    du[6] =                   + Fz

    du[7] = -mdot * τ
end

In [ ]:
p = SailorMoon.dyanmics_parameters()


θ0 = 0.0
one_year = 365.242 189 * 86400
ωb = 2*pi / (one_year / p.tstar)

param = [p.mu2, p.muS, p.as, θ0, p.oml, ωb 0.0, 0.0, 0.0, 0.0, 0.0]

In [ ]:
# initial condition
